In [ ]:
import torch
import gpytorch
import numpy as np
import time
import matplotlib.pyplot as plt

from torch import nn

from src import data_loader as dl
from src.data_loader import PVDataLoader
from src.beta_likelihood import BetaLikelihood_MeanParametrization

In [ ]:
# set seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# data parameters
DAY_INIT = 10
DAY_MIN = 8
DAY_MAX = 16
N_DAYS = 20
MINUTE_INTERVAL = 5
DAILY_DATA_POINTS = (DAY_MAX - DAY_MIN) * 60 / MINUTE_INTERVAL
N_HOURS_PRED = 2
N_SYSTEMS = 15
RADIUS = 0.35
COORDS = (55, -1.5)
IDX = 6

In [ ]:
loader = PVDataLoader(n_days=N_DAYS,
                    day_init=DAY_INIT,
                    n_systems=N_SYSTEMS,
                    radius=RADIUS,
                    coords=COORDS,
                    minute_interval=MINUTE_INTERVAL,
                    day_min=DAY_MIN,
                    day_max=DAY_MAX,
                    folder_name='pv_data',
                    file_name_pv='pv_data_clean.csv',
                    file_name_location='location_data_clean.csv')

x, y = loader.get_time_series()
y = y[:, IDX]
_, y_train, _, y_test = dl.train_test_split(x, y, n_hours=N_HOURS_PRED)


In [ ]:
if y.max() > 1:
    y[y > 1] = 1
# get time related variables
# periodic_time = dl.periodic_mapping(time, DAY_MIN, DAY_MAX, MINUTE_INTERVAL)
# x = torch.stack([time, periodic_time], dim=1)

# standardize input
x = (x - x.mean(dim=0)) / x.std(dim=0)

y_in = torch.zeros_like(x)

y_in[:len(y_train)] = y_train
y_in[len(y_train):] = torch.tensor(np.nan)


In [ ]:
# TODO run tests and make sure it works
# TODO add option for exogenous variables
# TODO check if this works for multivariate time series
# TODO set up for device agnostic code

class KalmanFilterSmoother(nn.Module):
    """ 
    Class for performing Kalman filter smoothing on 
    a linear Gaussian state space model.
    
    This is based on the work in: 
    https://ieeexplore.ieee.org/document/5589113

    Args:
        F (torch.Tensor): transition matrix
        Q (torch.Tensor): process noise covariance matrix
        H (torch.Tensor): observation matrix
        R (torch.Tensor): observation noise covariance matrix
    """
    def __init__(self, F, Q, H, R):
        self.F = F
        self.Q = Q
        self.H = H
        self.R = R

    def _predict(self, x, P):
        """ 
        Perform the prediction step of the Kalman filter.
        This should be called before the filtering update step.

        Args:
            x: torch.Tensor of shape (d,) where d is the spatial dimension
            P: torch.Tensor of shape (d, d)
        
        Returns:
            xnew: torch.Tensor of shape (d,) where d is the spatial dimension
            Pnew: torch.Tensor of shape (d, d)
        """

        F = self.F
        Q = self.Q
        return torch.matmul(F, x), torch.matmul(torch.matmul(F, P), F.t()) + Q

    def _filter_update(self, x, P, y):
        """ 
        Perform the filtering update step.
        This should be called after applying the forward pass.

        Args:
            x: torch.Tensor of shape (d,) where d is the spatial dimension
            P: torch.Tensor of shape (d, d)
            y: torch.Tensor of shape (d,) where d is the spatial dimension
        
        Returns:
            xnew: torch.Tensor of shape (d,) where d is the spatial dimension
            Pnew: torch.Tensor of shape (d, d)
        """

        R = self.R
        H = self.H
       
        # Compute Kalman gain matrix
       
        if not torch.isnan(y):
            S = H @ torch.linalg.matmul(P, H.t()) + R
            chol = torch.linalg.cholesky(S)
            Kt = torch.cholesky_solve(H @ P, chol)
            Hx = H @ x

            
            P - torch.matmul(torch.matmul(Kt, S.t()), Kt)
            torch.matmul(Kt, y - Hx).t()
       
            return x + torch.matmul(Kt, y - Hx).t(), P - torch.matmul(torch.matmul(Kt, S.t()), Kt)
       
        else:
            return x, P

    def _smoother_update(self, x_now, x_next, x_forecast, P_now, P_next, P_forecast):
        """ 
        Perform the smoothing update step. 
        This should be called after applying the forward pass.

        Args:
            x_now: torch.Tensor of shape (d,) where d is the spatial dimension
            x_next: torch.Tensor of shape (d,) where d is the spatial dimension
            x_forecast: torch.Tensor of shape (d,) where d is the spatial dimension
            P_now: torch.Tensor of shape (d, d)
            P_next: torch.Tensor of shape (d, d)
            P_forecast: torch.Tensor of shape (d, d)

        Returns:
            xnew: torch.Tensor of shape (d,) where d is the spatial dimension
            Pnew: torch.Tensor of shape (d, d)
        """

        F = self.F
        
        # Compute smoothing gain
        chol = torch.linalg.cholesky(P_forecast)
        Jt = torch.cholesky_solve(F @ P_now, chol)
        
        # Update
        xnew = x_now + torch.matmul(Jt, x_next - x_forecast).t()
        Pnew = P_now + torch.matmul(torch.matmul(Jt, P_next - P_forecast), Jt.t())
        
        return xnew, Pnew

    def forward_pass(self, x, P, y_list):
        """ 
        Calling the forward pass gives us the filtering distribution.
         Args:
            x: torch.Tensor of shape (d,) where d is the spatial dimension
            P: torch.Tensor of shape (d, d)
            y_list: torch.Tensor of shape (N, d) containing a list of observations at N timepoints.
                    Note that when there is no observation at time n, then set y_list[n] = torch.nan
        """
        means = []
        covariances = []
        
        for y in y_list:
            y.unsqueeze_(0)
            x, P = self._filter_update(x, P, y)
            
            means.append(x)
            covariances.append(P)
            
            x, P = self._predict(x, P)
       
        return torch.stack(means), torch.stack(covariances)

    def backward_pass(self, x, P):
        """ 
        Calling the backward pass gives us the smoothing distribution. This should be called after applying the forward pass.
        
        Args:
            x: torch.Tensor of shape (N, d) where N is the number of forward time steps and d is the spatial dimension
            P: torch.Tensor of shape (N, d, d)
        
        Returns:
            means: torch.Tensor of shape (N, d) where N is the number of forward time steps and d is the spatial dimension
            covariances: torch.Tensor of shape (N, d, d)
        """        
        N = x.shape[0]
       
        means = [x[-1]]
        covariances = [P[-1]]
        
        for n in range(N - 2, -1, -1):
            # Forecast
            xf, Pf = self._predict(x[n], P[n])
            # Update
            xnew, Pnew = self._smoother_update(x[n], x[n + 1], xf, P[n], P[n + 1], Pf)
            means.append(xnew)
            covariances.append(Pnew)
       
        return torch.flip(torch.stack(means), [0]), torch.flip(torch.stack(covariances), [0])

In [ ]:
delta_t = x[1] - x[0]
length_scale = 0.5
lmbda = np.sqrt(3) / length_scale
amplitude = 0.05

# set up state space model
A = torch.tensor([[0.,1.0],[-lmbda**2, -2*lmbda]], dtype=torch.float32)
L = torch.tensor([[0.],[1.]], dtype=torch.float32)
F = torch.eye(2) + delta_t * A
q = 4 * amplitude**2 * lmbda**3
Q = torch.tensor([[0.,0.],[0., q * delta_t]], dtype=torch.float32)
H = torch.tensor([[1., 0.]], dtype=torch.float32)
R = torch.tensor([[0.01]], dtype=torch.float32)

x0 = torch.tensor([0., 0.], dtype=torch.float32)
P = torch.tensor([[amplitude**2, 0.],[0., amplitude**2 * lmbda**2]], dtype=torch.float32)

f = KalmanFilterSmoother(F, Q, H, R)

In [ ]:
start = time.time()
x_pred, P_pred = f.forward_pass(x0, P, y_in)
stop = time.time()
print('Forward pass took {} seconds'.format(stop - start))